## Race Predictions ##
### An exploration of data forcasting techniques so I don't forgot how data works in python ###

In [2]:
import requests
import pandas as pd

#### Load Data ####

For the data, I will be using the  'jolpi.ca F1 API' which can be found [here](https://api.jolpi.ca/ergast/).

At first I played around with the idea of loading and structuing all the data myself, although this was far less performant and would require the split into testing and training data manually,
So instead I will be using Pandas.

In [7]:
# Output data in python dictionary
# # Get the bulk of the data as a python object
# race_years = ['2024', '2023', '2022']
# races = {}
# for year in race_years:
#     races[year] = {}

# # Pandas can just do this all automatically but good to know how it roughly works :)
# for year in race_years:
    
#     request_limiter = 0
#     results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100").json()["MRData"]
#     for result in results["RaceTable"]["Races"]:
#         races[year][result["raceName"]] = result["Results"]

#     while(int(results["total"]) > request_limiter + 100):
#         request_limiter += 100
#         results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100&offset="+str(request_limiter)).json()["MRData"]
#         for result in results["RaceTable"]["Races"]:  
#             races[year][result["raceName"]] = result["Results"]


'\n# Get the bulk of the data as a python object\nrace_years = [\'2024\', \'2023\', \'2022\']\nraces = {}\nfor year in race_years:\n    races[year] = {}\n\n# Pandas can just do this all automatically but good to know how it roughly works :)\nfor year in race_years:\n    \n    request_limiter = 0\n    results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100").json()["MRData"]\n    for result in results["RaceTable"]["Races"]:\n        races[year][result["raceName"]] = result["Results"]\n\n    while(int(results["total"]) > request_limiter + 100):\n        request_limiter += 100\n        results = requests.get("http://api.jolpi.ca/ergast/f1/"+year+"/results?limit=100&offset="+str(request_limiter)).json()["MRData"]\n        for result in results["RaceTable"]["Races"]:  \n            races[year][result["raceName"]] = result["Results"]\n'

In [14]:
# Output data in pandas dataframe
base_url = "http://api.jolpi.ca/ergast/f1/2024/results"
limit = 100
offset = 0

response = requests.get(f"{base_url}?limit={1}&offset={offset}").json()
total_results = int(response["MRData"]["total"])
race_dataframes = {}

while offset < total_results:
    response = requests.get(f"{base_url}?limit={limit}&offset={offset}").json()
    results = response["MRData"]["RaceTable"]["Races"]

    for race in results:
        race_name = race["raceName"] 
        if race_name not in race_dataframes:
            race_dataframes[race_name] = []

        for res in race["Results"]:
            race_dataframes[race_name].append({
                "DriverCode": res["Driver"]["code"],
                "ConstructorId": res["Constructor"]["constructorId"],
                "StartingPosition": res["grid"],
                "RaceResult": res["position"]
            })

    offset += limit

for race_name in race_dataframes:
    race_dataframes[race_name] = pd.DataFrame(race_dataframes[race_name])

for race_name, df in race_dataframes.items():
    print(f"Results for {race_name}:")
    print(df)


Results for Bahrain Grand Prix:
   DriverCode ConstructorId StartingPosition RaceResult
0         VER      red_bull                1          1
1         PER      red_bull                5          2
2         SAI       ferrari                4          3
3         LEC       ferrari                2          4
4         RUS      mercedes                3          5
5         NOR       mclaren                7          6
6         HAM      mercedes                9          7
7         PIA       mclaren                8          8
8         ALO  aston_martin                6          9
9         STR  aston_martin               12         10
10        ZHO        sauber               17         11
11        MAG          haas               15         12
12        RIC            rb               14         13
13        TSU            rb               11         14
14        ALB      williams               13         15
15        HUL          haas               10         16
16        OCO   